In [34]:
import pickle
from datetime import datetime
import re
import pandas as pd

In [35]:
brands = ["alcatel", "apple", "asus", "blu", "htc", "huawei", "infinix", "lenovo", "lg", "nokia",  "sony", "zte","xiaomi","samsung"]
data = []

In [36]:
for i in brands:
    with open(f'{i}.pkl','rb') as f:
        one_list = pickle.load(f)
        for j in one_list:
            j["brand"] = i
    print(i + " : " + str(len(one_list)))
    data.extend(one_list)

alcatel : 409
apple : 118
asus : 200
blu : 369
htc : 287
huawei : 441
infinix : 125
lenovo : 246
lg : 450
nokia : 300
sony : 158
zte : 369
xiaomi : 376
samsung : 850


In [37]:
def make_it_one_dictionary(one_device):
    sub_dict = {}

    for sub_dict_key, sub_dict_value in one_device.items():
        if isinstance(sub_dict_value, dict):
            if sub_dict_key in ['Main Camera', 'Selfie camera','Display','Battery','Tests']:
                for key, value in sub_dict_value.items():
                    new_key = f"{sub_dict_key} {key}"
                    sub_dict[new_key] = value
            else:
                for key, value in sub_dict_value.items():
                    sub_dict[key] = value
        else:
            sub_dict[sub_dict_key] = sub_dict_value
    if '\xa0' in sub_dict:
        del sub_dict['\xa0']

    return sub_dict

In [38]:
data_list = [make_it_one_dictionary(x) for x in data]

In [39]:
data[0].keys()

dict_keys(['phone_name', 'Network', 'Launch', 'Body', 'Display', 'Platform', 'Memory', 'Main Camera', 'Selfie camera', 'Sound', 'Comms', 'Features', 'Battery', 'Misc', 'brand'])

In [40]:
data[3].get("Main Camera")

{'Triple': '48 MP, f/1.8, 26mm (wide), 1/2.0", 0.8µm, PDAF\r\n2 MP, f/2.4, (macro)\r\n2 MP, f/2.4, (depth)',
 'Features': 'LED flash, HDR, panorama',
 'Video': '1080p@30fps'}

In [41]:
data_list[0]

{'phone_name': 'alcatel 1b (2022)',
 'Technology': 'GSM / HSPA / LTE',
 '2G bands': 'GSM 850 / 900 / 1800 / 1900 - SIM 1 & SIM 2 (Dual-SIM model only)',
 '3G bands': 'HSDPA 850 / 900 / 1900 / 2100 ',
 '4G bands': '1, 3, 5, 7, 8, 20, 28, 38, 40, 41 - 5031D, 5031G',
 'Speed': 'HSPA 42.2/11.1 Mbps, LTE Cat4 150/50 Mbps',
 'Announced': '2022, May 31',
 'Status': 'Available. Released 2022, May 31',
 'Dimensions': '146.7 x 71.9 x 10 mm (5.78 x 2.83 x 0.39 in)',
 'Weight': '172 g (6.07 oz)',
 'SIM': 'Single SIM (Nano-SIM) or Dual SIM (Nano-SIM, dual stand-by)',
 'Display Type': 'TFT LCD',
 'Display Size': '5.5 inches, 78.1 cm2 (~74.0% screen-to-body ratio)',
 'Display Resolution': '720 x 1440 pixels, 18:9 ratio (~293 ppi density)',
 'OS': 'Android 11 (Go edition)',
 'Chipset': 'Mediatek MT6761 Helio A22 (12 nm)',
 'CPU': 'Quad-core 2.0 GHz Cortex-A53',
 'GPU': 'PowerVR GE8300',
 'Card slot': 'microSDXC (dedicated slot)',
 'Internal': '32GB 2GB RAM',
 'Main Camera Single': '8 MP, f/2.0, 25 mm 

In [42]:
data[3].get("Selfie camera")

{'Single': '8 MP, f/2.0, 26mm (wide), 1/4", 1.12µm',
 'Features': 'HDR',
 'Video': '1080p@30fps'}

In [43]:
with open("data_list.pkl","wb") as f:
    pickle.dump(data_list,f)

In [44]:
df= pd.DataFrame(data_list)

In [45]:
df.to_csv("df.csv")

In [46]:
def year_rate(input_string):
    str_year= re.search(r'\b\d{4}\b',input_string)
    if str_year :
        return int(str_year.group(0))
    else : return None 

In [47]:
final = []
def Cleaning(z,row):
    ans = {}
    ans["name"] = row.get("phone_name")
    ans["brand"] = row.get("brand")
    net = ["2G","3G","4G","5G"]
    for i in net :
        for j in list(row.keys()):
            if i in j :
                ans[i] = True
                break
            else : ans[i] = False
    ans["Announced"] = year_rate(row.get("Announced"))

    ans["Status"] = row.get("Status").split()[0].replace(".", "")
    try:
        ans["Weight"] = float(row.get("Weight").split("g")[0])
    except : row["Weight"] = None
    try :
        ans["Length"] = (row["Dimensions"].split("x")[0])
        ans["Width"] = (row["Dimensions"].split("x")[1])
        ans["Diameter"] = (row["Dimensions"].split("x")[2].split("m")[0])
    except : 
        ans["Length"] = row["Width"] = row["Diameter"] = None
    sim = None
    try :
        SIM_CHECK = ['Mini-SIM' , 'Micro-SIM' , 'Nano-SIM' , 'eSIM','built-in']
        for sim_type in SIM_CHECK:
            if sim_type.lower() in row.get("SIM").lower():
                sim = sim_type
    except : 
        pass
    ans["SIM"] = sim
    ans["Display Type"] = row.get("Display Type")
    Display_Size = None
    try:
        Display_Size = row.get("Display Size").split()[0]
    except : pass
    ans["Display Size"] = Display_Size
    str_ppi = re.search(r'(\d+) ppi', row.get("Display Resolution"))
    ppi = None
    if str_ppi:
        ppi = int(str_ppi.group(1))
    ans["ppi"] = ppi
    str_ratio = re.search(r'(\d+(\.\d+)?)%', row.get("Display Size"))
    ratio = None
    if str_ratio:
        ratio = float(str_ratio.group(1))
    ans["body ratio"] = ratio
    os = None
    try:
    #    os = row.get("OS").split(",")[0]
        str_os = re.search(r'([a-zA-Z]+\s*\d+(\.\d+)*)',row.get("OS"))
        str_os2 = re.search(r'(\b\S+\s+OS\b)',row.get("OS"))
        if str_os:
            os = str_os.group(1)
        elif str_os2:
            os = str_os2.group(1)
    except :
        pass
    ans["OS"] = os
    str_battery_capacity = re.search(r'(\d+)\s*mAh',row.get("Battery Type"))
    battery_capacity = None
    if str_battery_capacity:
        battery_capacity = int(str_battery_capacity.group(1)) 
    ans["battery_capacity"] = battery_capacity
    price_patterns = [
    r'€\s*([\d,.]+)',
    r'About\s+(\d+)\s+EUR']

    price = None

    if 'Price' in row:
        for pattern in price_patterns:
            match = re.search(pattern, row.get("Price"))
            if match:
                price = float(match.group(1).replace(",", ""))
                break  # Break once a match is found to avoid duplicate entries
    ans["Price"] = price
    cpu = None
    try:
        cpu = row.get("CPU").split()[0]
    except:
        pass
    ans["CPU"] = cpu
    ratio = None
    str_Ratio = re.search(r'(\d+:\d+)\s*ratio',row.get("Display Resolution"))
    if str_Ratio :
        ratio = str_Ratio.group(1)
    ans["ratio"] = ratio
    pixel = None
    str_pixel = re.search(r'(\d+\s*x\s*\d+)\s*pixels',row.get("Display Resolution"))
    if str_pixel:
        ans["pixel"] = int(str_pixel.group(1).split("x")[0]) * int(str_pixel.group(1).split("x")[1])
    ans["WLAN"] = row.get("WLAN")
    ans["Colors"] = row.get("Colors")
    ans["Sensors"] = row.get("Sensors")
    ans["Bluetooth"] = row.get("Bluetooth")
    ans["GPU"] = row.get("GPU")
    if row.get("Loudspeaker ") == "Yes" :
        ans["Loudspeaker"] = True
    else :
        ans["Loudspeaker"] = False
    if row.get("3.5mm jack ") == "Yes" :
        ans["3.5mm jack"] = True
    else :
        ans["3.5mm jack"] = False
    ans["Chipset"] = row.get("Chipset")
    ans["Network"] = str(row.get("Technology"))+" " + str(row.get("Speed"))
    ans["Internal"] = row.get("Internal")
    ans["Card slot"] = row.get("Card slot")

    ans["Main Camera"] = data[z].get("Main Camera")
    ans["Selfie camera"] = data[z].get("Selfie camera")

    final.append(ans)
    

In [48]:
for i , x in enumerate(data_list):
    Cleaning(i,x)
    
    

In [49]:
df_final = pd.DataFrame(final)

In [50]:
df_final["Length"] = pd.to_numeric(df_final["Length"],errors="coerce")
df_final["Width"] = pd.to_numeric(df_final["Width"],errors="coerce")
df_final["Diameter"] = pd.to_numeric(df_final["Diameter"],errors="coerce")

In [51]:
def extract_gb_numbers(text):
    if text is None:
        return [None, None]
    patern = [r'(\d+GB) (\d+MB)',r'(\d+GB) (\d+GB)',r'(\d+GB) (\d+\.\d+GB)']
    for i in patern :
        numbers = re.search(i, text.split(",")[0])
        if numbers :
            return [numbers.group(1),numbers.group(2)]  
    return [None, None]

In [58]:
df_final

,name,brand,2G,3G,4G,5G,Announced,Status,Weight,Length,...,GPU,Loudspeaker,3.5mm jack,Chipset,Network,Card slot,Main Camera,Selfie camera,Storage,RAM
0,alcatel 1b (2022),alcatel,True,True,True,False,2022.0,Available,172.0,146.7,...,PowerVR GE8300,True,True,Mediatek MT6761 Helio A22 (12 nm),"GSM / HSPA / LTE HSPA 42.2/11.1 Mbps, LTE Cat4...",microSDXC (dedicated slot),"{'Single': '8 MP, f/2.0, 25 mm (wide), 1/4.0"",...","{'Single': '5 MP, f/2.2, 24 mm (wide), 1/5"", 1...",32GB,2GB
1,alcatel 1l pro (2021),alcatel,True,True,True,False,2021.0,Available,190.0,156.4,...,IMG8322,True,True,Unisoc SC9863A (28nm),"GSM / HSPA / LTE HSPA 21.1/5.76 Mbps, LTE Cat4...",microSDXC (uses shared SIM slot),"{'Dual': '13 MP, f/2.2, 26mm (wide), 1/3.06"", ...","{'Single': '5 MP, f/2.2, 27mm (wide), 1/5.0"", ...",32GB,2GB
2,alcatel 1 (2021),alcatel,True,True,True,False,2021.0,Available,134.0,137.6,...,PowerVR GE8100,True,True,Mediatek MT6739 (28 nm),"GSM / HSPA / LTE HSPA 42.2/11.5 Mbps, LTE Cat4...",microSDHC (dedicated slot),"{'Single': '5 MP, f/2.4, 1/5.0"", 1.12µm', 'Fea...","{'Single': '2 MP, f/2.8', 'Video': '720p@30fps'}",8GB,1GB
3,alcatel 3l (2021),alcatel,True,True,True,False,2021.0,Available,194.0,165.6,...,PowerVR GE8320,True,True,Mediatek MT6762D Helio P22 (12 nm),"GSM / HSPA / LTE HSPA 42.2/5.76 Mbps, LTE Cat4...",microSDXC (only single sim model),"{'Triple': '48 MP, f/1.8, 26mm (wide), 1/2.0"",...","{'Single': '8 MP, f/2.0, 26mm (wide), 1/4"", 1....",64GB,4GB
4,alcatel 1s (2021),alcatel,True,True,True,False,2021.0,Available,190.0,165.6,...,PowerVR GE8320,True,True,Mediatek MT6762D Helio P22 (12 nm),"GSM / HSPA / LTE HSPA 42.2/5.76 Mbps, LTE Cat4...",microSDXC (uses shared SIM slot),"{'Triple': '13 MP, f/2.0, 27mm (wide), 1/3.06""...","{'Single': '5 MP, f/2.2, 27mm (wide), 1/5"", 1....",32GB,3GB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4693,samsung s7070 diva,samsung,True,False,False,False,2009.0,Discontinued,94.0,101.0,...,None,True,False,None,GSM None,microSDHC (dedicated slot),"{'Single': '3.15 MP, AF', 'Video': '320p'}",{' ': 'No'},None,None
4694,samsung c3730c,samsung,True,True,False,False,2012.0,Discontinued,99.5,104.0,...,None,True,False,None,GSM HSPA,microSD (dedicated slot),"{'Single': '1.3 MP', 'Video': 'QCIF@15fps'}",{' ': 'No'},None,None
4695,samsung s5150 diva folder,samsung,True,False,False,False,2009.0,Discontinued,82.8,94.0,...,None,True,False,None,GSM None,microSDHC (dedicated slot),"{'Single': '3.15 MP, AF', 'Video': '320p'}",{' ': 'No'},None,None
4696,samsung c5130,samsung,True,True,False,False,2009.0,Discontinued,100.0,97.5,...,None,True,False,None,"GSM / UMTS Yes, 384 kbps",microSDHC (dedicated slot),"{'Single': '1.3 MP', 'Video': 'QCIF@15fps'}","{'Single': 'VGA', 'Video': ''}",None,None


In [52]:
df_final[["Storage" , "RAM"]] = pd.DataFrame(df_final["Internal"].apply(extract_gb_numbers).to_list(),index=df_final.index)

In [53]:
df_final.drop(columns=["Internal"],inplace=True)

In [54]:
def correct_cpu(text) :
    core_cpu ={"Quad-core" : 4 , "Octa-core" : 8 , "Dual-core" : 2 ,"Deca-core" : 10 , "Hexa-core" : 6 , "Triple-core" : 3}
    if text in core_cpu.keys() :
        return core_cpu.get(text)
    return None

In [55]:
df_final["CPU"] = df_final["CPU"].apply(correct_cpu)

In [56]:
df_final.dropna().to_csv("data_with_out_null.csv")

In [57]:
df_final.to_csv("data.csv")